In [1]:
using MLDatasets
using Statistics
using PyPlot
using Distributions
using ProgressMeter
using Random
using MLBase

In [2]:
# load full training set
train_x, train_y = MNIST.traindata()

# load full test set
test_x,  test_y  = MNIST.testdata()

xtrain0 = zeros(784,12000)
xtrain1 = zeros(784,12000)
xlabel =zeros(12000)
global counter0=0
global counter1=0
print(size(train_x,3))
for i=1:size(train_x,3)
    if train_y[i]==7
        global counter0=counter0+1
        xtrain0[:,counter0]=vec(1.0*train_x[:,:,i])
    end
    if train_y[i]==9
        global counter1=counter1+1
        xtrain1[:,counter1]=vec(1.0*train_x[:,:,i])
    end            
end
xtest0 = zeros(784,12000)
xtest1 = zeros(784,12000)
xtestlabel =zeros(12000)
global counter0=0
global counter1=0
for i=1:size(test_x,3)
    if test_y[i]==7
        global counter0=counter0+1
        xtest0[:,counter0]=vec(1.0*test_x[:,:,i])
    end
    if test_y[i]==9
        global counter1=counter1+1
        xtest1[:,counter1]=vec(1.0*test_x[:,:,i])
    end            
end

60000

In [3]:
Ntrain=5900*2; Ntest=1000*2

x=[xtrain0[:,1:Int(Ntrain/2)] xtrain1[:,1:Int(Ntrain/2)]]
c=ones(Int,Ntrain)
for i=1:Int(Ntrain/2)
    c[i]=0
end

xtest=[xtest0[:,1:Int(Ntest/2)] xtest1[:,1:Int(Ntest/2)]]
ctest=ones(Int,Ntest)
for i=1:Int(Ntest/2)
    ctest[i]=0
end

In [4]:
function loglik(theta,c,x)
    L=0.0
    for n=1:size(x,2)
        L = L+log(sigmoid( (2*c[n]-1).*sum(theta.*x[:,n])))    
    end
    return L
end

loglik (generic function with 1 method)

In [5]:
function plot_image(x)
    xx=reshape(x,28,28)
    xxx=zeros(28,28)
    for i=1:28
        for j=1:28
            xxx[29-i,j]=xx[j,i]
        end
    end
    pcolormesh(xxx,cmap="gray")
    axis("equal")
    gca()[:set_axis_off]()
end

plot_image (generic function with 1 method)

In [6]:
function grad_theta(theta,c,x)
    g=zeros(size(x))
    cbar=2*c-1
    g = cbar*(1-sigmoid(cbar.*sum(theta.*x))).*x
end

grad_theta (generic function with 1 method)

In [7]:
function prob_cgtc(epsilon,tc)
    if epsilon>10
        p0g0=1 #otherwise numerical error
    else
        p0g0=exp(epsilon)/(exp(epsilon)+1)
    end
    p1g1=p0g0
    p1g0=1-p0g0
    p0g1=1-p1g1    
    qcgtc = zeros(2,1)
    if tc==0
        qcgtc[1] = p0g0
        qcgtc[2] = p0g1
    else
        qcgtc[1] = p1g0
        qcgtc[2] = p1g1
    end
    qcgtc=qcgtc./sum(qcgtc)
end

function prob_xgtx_binomial(tx,b,m,epsilon)
    sigma = m/(epsilon)
    theta = 0.11940392323596105
    thetaB = zeros(length(tx))
    for i=1:length(tx)
        A = exp(-abs(tx[i]-1)/sigma)*theta
        B = exp(-abs(tx[i])/sigma)*(1-theta)
        thetaB[i] = A/(A+B)
    end    
    return thetaB
end

prob_xgtx_binomial (generic function with 1 method)

In [8]:
function pixelization(x,b,m,epsilon,bounds=false)
    d = convert(Int64,28/b)
    Pb = zeros(d,d)
    x_tilde = zeros(784)
    x_pixel = zeros(convert(Int64,784/b^2))
    for i=1:784
        if rand()<0.5
            x_tilde[i]=x[i]+m/(epsilon)*randexp()
        else
            x_tilde[i]=x[i]-m/(epsilon)*randexp()
        end
    end
    x_pixel=x_tilde
    return (x_tilde,x_pixel)    
end

pixelization (generic function with 2 methods)

In [9]:
# SPREAD DIVERGENCE

#prior_mean=mean(x,dims=2)
#prior_var=std(x,dims=2)
#spread_std=std(x,dims=2) .+ 0.01
#spread_var=spread_std.^2

# generate the noisy training data
function spread_divergence(c,x,b,m,epsilon,epsilon_b,bounds=true)
    if epsilon_b>9
        p0g0=1 #otherwise numerical error
    else
        p0g0=exp(epsilon_b)/(exp(epsilon_b)+1)
    end    
    
    p1g1=p0g0
    p1g0=1-p0g0
    p0g1=1-p1g1

    ctilde=zeros(Int,size(c))
    xtilde=zeros(size(x))
    x_pixel=zeros(784,size(x)[2])
    for n=1:size(x)[2]
        if c[n]==0
            if rand()<p0g0
                ctilde[n]=0
            else
                ctilde[n]=1
            end
        end
        if c[n]==1
            if rand()<p1g1
                ctilde[n]=1
            else
                ctilde[n]=0
            end
        end
        (xtilde[:,n],x_pixel[:,n])=pixelization(x[:,n],b,m,epsilon,bounds)
    end
    return (ctilde, xtilde, x_pixel)
end

spread_divergence (generic function with 2 methods)

In [10]:
sigmoid(x) = 1.0 ./ (1.0 .+exp.(-x))

function truncate_bound(tx)
    x_truncated=zeros(length(tx))
    for i=1:length(tx)
        x_truncated[i] = tx[i]
        if tx[i] > 1
            x_truncated[i] = 1
        end
        if tx[i] < 0
            x_truncated[i] = 0
        end
    end
    return x_truncated
end
# function logistic_regression(Nloops,lr,lambda,x_train,c_train)
#     theta=zeros(784)
#     learning_rate=lr
#     LL=zeros(Nloops)
#     train_size=length(c_train)
#     prev_loss=1e6
    
#     for loop=1:Nloops 
#         grad=zeros(784)
#         for n=1:train_size
#             grad=grad + grad_theta(theta,c_train[n],x_train[:,n])
#         end
#         theta = theta+learning_rate*(grad/train_size-lambda*theta)
#         if loop%100==0
#             llk = loglik(theta,c_train,x_train)
#             L2norm_sqrd = sum(theta.^2)
#             loss = -llk+0.5*lambda*L2norm_sqrd
#             println("loop= $(loop) llk = $(round(llk; digits=2)) L2norm^2 = $(round(L2norm_sqrd; digits=4)) loss = $(round(loss; digits=2))")
#             if (prev_loss-loss)<3 && loop>199#stopping criteria
#                 break
#             else
#                 prev_loss=loss
#             end
#         end
#     end
#     return theta
# end

truncate_bound (generic function with 1 method)

In [11]:
b=1
m=784
S=16 # number of spread samples per training datapoint
update = 1
fold=10
X = 784
# for epsilon in (1,3,10,30,100,300,1e3,3e3,1e4,3e4,1e5)
for epsilon in (10)
    train_index=collect(Kfold(5900*2,10))
    train_acc,valid_acc,test_acc=zeros(fold),zeros(fold),zeros(fold)
    for epsilon_b in (3,1)
        for i=1:2
            println("fold = $(i) epsilon = $(epsilon) epsilon_b=$(epsilon_b) flip=$(1/(exp(epsilon_b)+1))")
        
            train_size = length(train_index[i])
            valid_size = 5900*2-train_size
            x_train = zeros(784,train_size)
            x_valid = zeros(784,valid_size)
            for j=1:train_size
                x_train[:,j] = x[:,train_index[i][j]]
            end

            c_train = [c[x] for x in train_index[i]]
            valid_index = filter!(x->x∉train_index[i],[x for x=1:5900*2])
            for j=1:valid_size
                x_valid[:,j] = x[:,valid_index[j]]
            end        

            c_valid = [c[x] for x in valid_index]
            Nloops=100
            (ctilde, xtilde, x_pixel)=spread_divergence(c_train,x_train,b,m,epsilon,epsilon_b,false)
            thetaS=zeros(X)

            LLS_valid=zeros(Nloops÷update)
            LLS_test=zeros(Nloops÷update)
            thetaB=zeros(size(x_pixel))

            learning_rate=0.3
            
            for n=1:train_size        # speed-up: only find thetaB once
                thetaB[:,n] = prob_xgtx_binomial(x_pixel[:,n],b,m,epsilon)
            end
        
            grad_old=zeros(X)
            @time  for loop=1:Nloops
                grad_new=zeros(X)
                grad=zeros(X)
                for n=1:train_size
                    pc=prob_cgtc(epsilon_b,ctilde[n])# second element is prob that c=1
                    w=zeros(S)
                    thisgrad=zeros(X)
                    for s=1:S
                        xsample=zeros(X)
                        csample=rand(Binomial(1,pc[2])) # sample p(c|ctilde)
                        for l=1:X
                            xsample[l]=rand(Binomial(1,thetaB[l,n]))
                        end                  
                        w[s]=sigmoid((2*csample-1)*sum(thetaS.*xsample))
                        thisgrad=thisgrad + w[s]*grad_theta(thetaS,csample,xsample)
                    end
                    grad=grad + thisgrad./sum(w)
                end
                if loop==1
                    grad_new=grad
                else
                    grad_new=0.8*grad_old+0.2*grad
                end
                grad_old=grad_new
                thetaS = thetaS+learning_rate*grad_new/train_size#momentum
                if loop%update==0
                    ctestpred=zeros(Int,Ntest)
                    for n=1:Ntest
                        if sigmoid(sum(thetaS.*xtest[:,n])) >0.5
                            ctestpred[n]=1
                        end    
                    end
                    test_accuracy = mean(ctestpred.==ctest)

                    cvalidpred=zeros(Int,valid_size)
                    for n=1:valid_size
                        if sigmoid(sum(thetaS.*x_valid[:,n])) >0.5
                            cvalidpred[n]=1
                        end    
                    end
                    valid_accuracy = mean(cvalidpred.==c_valid)

                    LLS_valid[loop÷update]=loglik(thetaS,c_valid,x_valid)
                    LLS_test[loop÷update]=loglik(thetaS,ctest,xtest)
                    println("loop=$(loop) Valid LL=$(round(LLS_valid[loop÷update]; digits=2)) Valid accuracy = $(round(valid_accuracy; digits=3)) Test LL=$(round(LLS_test[loop÷update]; digits=2)) Test Accuracy=$(round(test_accuracy; digits=3)))")
                end             
            end
        end
    end
end


fold = 1 epsilon = 10 epsilon_b=3 flip=0.04742587317756678
loop=1 Valid LL=-817.97 Valid accuracy = 0.487 Test LL=-1386.29 Test Accuracy=0.5)
loop=2 Valid LL=-817.99 Valid accuracy = 0.487 Test LL=-1386.23 Test Accuracy=0.5)
loop=3 Valid LL=-818.03 Valid accuracy = 0.487 Test LL=-1386.24 Test Accuracy=0.5)
loop=4 Valid LL=-818.09 Valid accuracy = 0.487 Test LL=-1386.3 Test Accuracy=0.5)
loop=5 Valid LL=-818.14 Valid accuracy = 0.487 Test LL=-1386.4 Test Accuracy=0.5)
loop=6 Valid LL=-818.2 Valid accuracy = 0.487 Test LL=-1386.52 Test Accuracy=0.5)
loop=7 Valid LL=-818.22 Valid accuracy = 0.487 Test LL=-1386.59 Test Accuracy=0.5)
loop=8 Valid LL=-818.25 Valid accuracy = 0.485 Test LL=-1386.69 Test Accuracy=0.5)
loop=9 Valid LL=-818.28 Valid accuracy = 0.485 Test LL=-1386.79 Test Accuracy=0.499)
loop=10 Valid LL=-818.31 Valid accuracy = 0.486 Test LL=-1386.9 Test Accuracy=0.492)
loop=11 Valid LL=-818.35 Valid accuracy = 0.469 Test LL=-1387.01 Test Accuracy=0.476)
loop=12 Valid LL=-818.38

loop=96 Valid LL=-819.84 Valid accuracy = 0.299 Test LL=-1389.76 Test Accuracy=0.286)
loop=97 Valid LL=-819.8 Valid accuracy = 0.305 Test LL=-1389.72 Test Accuracy=0.296)
loop=98 Valid LL=-819.8 Valid accuracy = 0.305 Test LL=-1389.75 Test Accuracy=0.304)
loop=99 Valid LL=-819.81 Valid accuracy = 0.306 Test LL=-1389.8 Test Accuracy=0.31)
loop=100 Valid LL=-819.81 Valid accuracy = 0.314 Test LL=-1389.81 Test Accuracy=0.318)
23585.192175 seconds (49.55 G allocations: 1.827 TiB, 0.86% gc time)
fold = 2 epsilon = 10 epsilon_b=3 flip=0.04742587317756678
loop=1 Valid LL=-817.69 Valid accuracy = 0.504 Test LL=-1385.92 Test Accuracy=0.5)
loop=2 Valid LL=-817.5 Valid accuracy = 0.504 Test LL=-1385.61 Test Accuracy=0.5)
loop=3 Valid LL=-817.34 Valid accuracy = 0.504 Test LL=-1385.35 Test Accuracy=0.5)
loop=4 Valid LL=-817.21 Valid accuracy = 0.504 Test LL=-1385.15 Test Accuracy=0.5)
loop=5 Valid LL=-817.12 Valid accuracy = 0.504 Test LL=-1385.0 Test Accuracy=0.5)
loop=6 Valid LL=-817.1 Valid acc

loop=90 Valid LL=-817.41 Valid accuracy = 0.505 Test LL=-1384.98 Test Accuracy=0.504)
loop=91 Valid LL=-817.42 Valid accuracy = 0.504 Test LL=-1384.98 Test Accuracy=0.503)
loop=92 Valid LL=-817.44 Valid accuracy = 0.504 Test LL=-1385.0 Test Accuracy=0.503)
loop=93 Valid LL=-817.5 Valid accuracy = 0.503 Test LL=-1385.07 Test Accuracy=0.503)
loop=94 Valid LL=-817.53 Valid accuracy = 0.503 Test LL=-1385.11 Test Accuracy=0.502)
loop=95 Valid LL=-817.55 Valid accuracy = 0.503 Test LL=-1385.15 Test Accuracy=0.502)
loop=96 Valid LL=-817.59 Valid accuracy = 0.503 Test LL=-1385.21 Test Accuracy=0.502)
loop=97 Valid LL=-817.62 Valid accuracy = 0.503 Test LL=-1385.28 Test Accuracy=0.502)
loop=98 Valid LL=-817.64 Valid accuracy = 0.503 Test LL=-1385.34 Test Accuracy=0.502)
loop=99 Valid LL=-817.68 Valid accuracy = 0.503 Test LL=-1385.43 Test Accuracy=0.502)
loop=100 Valid LL=-817.69 Valid accuracy = 0.504 Test LL=-1385.45 Test Accuracy=0.501)
21784.006977 seconds (49.55 G allocations: 1.827 TiB, 0

loop=86 Valid LL=-816.89 Valid accuracy = 0.487 Test LL=-1383.49 Test Accuracy=0.5)
loop=87 Valid LL=-816.85 Valid accuracy = 0.487 Test LL=-1383.42 Test Accuracy=0.5)
loop=88 Valid LL=-816.82 Valid accuracy = 0.487 Test LL=-1383.34 Test Accuracy=0.5)
loop=89 Valid LL=-816.8 Valid accuracy = 0.487 Test LL=-1383.28 Test Accuracy=0.5)
loop=90 Valid LL=-816.76 Valid accuracy = 0.487 Test LL=-1383.19 Test Accuracy=0.501)
loop=91 Valid LL=-816.74 Valid accuracy = 0.487 Test LL=-1383.12 Test Accuracy=0.5)
loop=92 Valid LL=-816.72 Valid accuracy = 0.487 Test LL=-1383.05 Test Accuracy=0.5)
loop=93 Valid LL=-816.73 Valid accuracy = 0.487 Test LL=-1383.05 Test Accuracy=0.5)
loop=94 Valid LL=-816.72 Valid accuracy = 0.487 Test LL=-1383.0 Test Accuracy=0.5)
loop=95 Valid LL=-816.69 Valid accuracy = 0.487 Test LL=-1382.93 Test Accuracy=0.5)
loop=96 Valid LL=-816.61 Valid accuracy = 0.487 Test LL=-1382.8 Test Accuracy=0.5)
loop=97 Valid LL=-816.55 Valid accuracy = 0.487 Test LL=-1382.72 Test Accurac

loop=81 Valid LL=-817.92 Valid accuracy = 0.476 Test LL=-1385.99 Test Accuracy=0.495)
loop=82 Valid LL=-817.99 Valid accuracy = 0.481 Test LL=-1386.11 Test Accuracy=0.498)
loop=83 Valid LL=-818.05 Valid accuracy = 0.479 Test LL=-1386.19 Test Accuracy=0.498)
loop=84 Valid LL=-818.13 Valid accuracy = 0.484 Test LL=-1386.34 Test Accuracy=0.497)
loop=85 Valid LL=-818.21 Valid accuracy = 0.486 Test LL=-1386.48 Test Accuracy=0.497)
loop=86 Valid LL=-818.3 Valid accuracy = 0.492 Test LL=-1386.65 Test Accuracy=0.494)
loop=87 Valid LL=-818.43 Valid accuracy = 0.493 Test LL=-1386.86 Test Accuracy=0.496)
loop=88 Valid LL=-818.52 Valid accuracy = 0.494 Test LL=-1387.01 Test Accuracy=0.5)
loop=89 Valid LL=-818.63 Valid accuracy = 0.498 Test LL=-1387.2 Test Accuracy=0.5)
loop=90 Valid LL=-818.76 Valid accuracy = 0.497 Test LL=-1387.43 Test Accuracy=0.498)
loop=91 Valid LL=-818.89 Valid accuracy = 0.496 Test LL=-1387.64 Test Accuracy=0.498)
loop=92 Valid LL=-818.95 Valid accuracy = 0.496 Test LL=-138

In [ ]:
b=1
m=784
S=16 # number of spread samples per training datapoint
update = 1
fold=10


X = 784
#     for epsilon in (1,3,5,7,10,20)
# for epsilon in (1,3,10,30,100,300,1e3,3e3,1e4,3e4,1e5)
for epsilon in (1e3)
    train_index=collect(Kfold(5900*2,10))
    train_acc,valid_acc,test_acc=zeros(fold),zeros(fold),zeros(fold)
    for i=1:fold
        println("fold = $(i) epsilon = $(epsilon)")
        train_size = length(train_index[i])
        valid_size = 5900*2-train_size
        x_train = zeros(784,train_size)
        x_valid = zeros(784,valid_size)
        for j=1:train_size
            x_train[:,j] = x[:,train_index[i][j]]
        end
        
        c_train = [c[x] for x in train_index[i]]
        valid_index = filter!(x->x∉train_index[i],[x for x=1:5900*2])
        for j=1:valid_size
            x_valid[:,j] = x[:,valid_index[j]]
        end        
        
        c_valid = [c[x] for x in valid_index]
        
        (ctilde, xtilde, x_pixel)=spread_divergence(c_train,x_train,b,m,epsilon,false)
        thetaS=zeros(X)
        
        Nloops=1000
        LLS_valid=zeros(Nloops÷update)
        LLS_test=zeros(Nloops÷update)
        thetaB=zeros(size(x_pixel))
        
        
        ########
#         # Pre-train
#         learning_rate=0.2
#         Nloops=300
# #         @showprogress 1 "Standard Logistic Regression training"  
#         for loop=1:Nloops 
#             grad=zeros(X)
#             x_truncate=zeros(X,train_size)
#             for n=1:train_size
#                 x_truncate[:,n]=truncate_bound(x_pixel[:,n])
#                 grad=grad + grad_theta(thetaS,ctilde[n],x_truncate[:,n])
#             end
#             thetaS = thetaS+learning_rate*grad/train_size
#         end
        
        #####################
        learning_rate=0.3
        Nloops=200
        for n=1:train_size        # speed-up: only find thetaB once
            thetaB[:,n] = prob_xgtx_binomial(x_pixel[:,n],b,m,epsilon)
        end
        grad_old=zeros(X)
        @time  for loop=1:Nloops
            grad_new=zeros(X)
            grad=zeros(X)
            for n=1:train_size
                pc=prob_cgtc(epsilon,ctilde[n])# second element is prob that c=1
                w=zeros(S)
                thisgrad=zeros(X)
                for s=1:S
                    xsample=zeros(X)
                    csample=rand(Binomial(1,pc[2])) # sample p(c|ctilde)
                    for l=1:X
                        xsample[l]=rand(Binomial(1,thetaB[l,n]))
                    end                  
                    w[s]=sigmoid((2*csample-1)*sum(thetaS.*xsample))
                    thisgrad=thisgrad + w[s]*grad_theta(thetaS,csample,xsample)
                end
                grad=grad + thisgrad./sum(w)
            end
            if loop==1
                grad_new=grad
            else
                grad_new=0.9*grad_old+0.1*grad
            end
            grad_old=grad_new
            thetaS = thetaS+learning_rate*grad_new/train_size#momentum
            if loop%update==0
                ctestpred=zeros(Int,Ntest)
                for n=1:Ntest
                    if sigmoid(sum(thetaS.*xtest[:,n])) >0.5
                        ctestpred[n]=1
                    end    
                end
                test_accuracy = mean(ctestpred.==ctest)
                
                cvalidpred=zeros(Int,valid_size)
                for n=1:valid_size
                    if sigmoid(sum(thetaS.*x_valid[:,n])) >0.5
                        cvalidpred[n]=1
                    end    
                end
                valid_accuracy = mean(cvalidpred.==c_valid)
            
                LLS_valid[loop÷update]=loglik(thetaS,c_valid,x_valid)
                LLS_test[loop÷update]=loglik(thetaS,ctest,xtest)
                println("loop=$(loop) Valid LL=$(round(LLS_valid[loop÷update]; digits=2)) Valid accuracy = $(round(valid_accuracy; digits=3)) Test LL=$(round(LLS_test[loop÷update]; digits=2)) Test Accuracy=$(round(test_accuracy; digits=3)))")
            end             
        end
        
# #             check validation data accuracy
#         cvalidpred=zeros(Int,valid_size)
#         for n=1:valid_size
#             if sigmoid(sum(thetaS.*x_valid[:,n])) >0.5
#                 cvalidpred[n]=1
#             end    
#         end
#         valid_acc[i] = mean(cvalidpred.==c_valid)
#         println("validation accuracy = $(round(valid_acc[i]; digits=3))")

#         ctestpred=zeros(Int,Ntest)
#         for n=1:Ntest
#             if sigmoid(sum(thetaS.*xtest[:,n])) >0.5
#                 ctestpred[n]=1
#             end    
#         end
#         test_acc[i] = mean(ctestpred.==ctest)   
#         println("test accuracy = $(round(test_acc[i]; digits=3))")  
    
#         print("Train Log-likelihood, ", LLS_valid) 
#         print("Test Log-likelihood, ", LLS_test) 
    end
#     println("====================================================")
#     println("Summary: e = $(epsilon) flip = $(round(1/(exp(epsilon)+1); digits=3))") 
#     println("validation accuracy = $(round(mean(valid_acc); digits=3))") 
#     println("test accuracy = $(round(mean(test_acc); digits=3))")
#     println("====================================================")
end


over-train,need regularization??